### dev_notebook_

This is your development environment...you may start:
- Exploring your data.
- Testing the ad-hoc methods.
- Designing your pipeline.

In [1]:
import requests
import pandas as pd 
import numpy as np
import argparse
from scipy.spatial.distance import cdist
from math import radians, cos, sin, asin, sqrt

In [2]:
base_url = "https://datos.madrid.es/egob"
defined_json = "/catalogo/202162-0-instalaciones-accesibles-municip.json"

In [3]:
key_places = pd.json_normalize(requests.get(base_url + defined_json).json()["@graph"])
key_places

,@id,id,title,relation,address.district.@id,address.area.@id,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,organization.organization-desc,organization.accesibility,organization.schedule,organization.services,organization.organization-name,@type
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,12046758,Centro de acogida 'Juan Luis Vives',http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28052,CALLE ALCALDE JUAN DE MATA SEVILLANO 18,40.398796,-3.584448,Información Complementaria: Plazas: 140...,1,"Servicio ininterrumpido, 24 horas los 365 días...","Alojamiento, Manutención: desayuno, comida, me...",Centro de acogida 'Juan Luis Vives',NaN
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,11867687,Departamento de Protección a la Infancia y Ado...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28045,PASEO CHOPERA 41 PLANTA Baja,40.394160,-3.699021,"Metro: Legazpi (línea 3 y 6)Bus: 6, 8, 18, ...",1,De lunes a viernes de 9 a 14 horas.,,Departamento de Protección a la Infancia y Ado...,https://datos.madrid.es/egob/kos/entidadesYorg...
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,12040036,Grupo Municipal Más Madrid,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28013,CALLE MAYOR 71 PLANTA 1ª y 2ª,40.415265,-3.711579,Portavoz: Rita Maestre Fernández Portavoz adj...,1,,,Grupo Municipal Más Madrid,https://datos.madrid.es/egob/kos/entidadesYorg...
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,12040015,Grupo Municipal Socialista de Madrid,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28013,CALLE MAYOR 71 PLANTA 3ª y 4ª,40.415265,-3.711579,Portavoz: M&amp;ordf; Reyes Maroto Illera Por...,1,"De 9 a 14 h, previa cita concertada en el telé...",,Grupo Municipal Socialista de Madrid,https://datos.madrid.es/egob/kos/entidadesYorg...
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,12040068,Grupo Municipal del Partido Popular,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28005,PLAZA VILLA 5,40.415199,-3.710611,Portavoz: Carlos Izquierdo Torres Portavoz a...,1,,,Grupo Municipal del Partido Popular,https://datos.madrid.es/egob/kos/entidadesYorg...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,https://datos.madrid.es/egob/catalogo/tipo/ent...,213,Área de Gobierno de Hacienda y Personal,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28014,CALLE ALCALA 45,40.419110,-3.696280,Titular del Área: Engracia Hidalgo Tena Com...,1,,,Área de Gobierno de Hacienda y Personal,https://datos.madrid.es/egob/kos/entidadesYorg...
1232,https://datos.madrid.es/egob/catalogo/tipo/ent...,8095780,Área de Gobierno de Obras y Equipamientos,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28014,CALLE ALCALA 45 PLANTA 3,40.419110,-3.696280,Titular del Área: Paloma García Romero. Comp...,1,,,Área de Gobierno de Obras y Equipamientos,https://datos.madrid.es/egob/kos/entidadesYorg...
1233,https://datos.madrid.es/egob/catalogo/tipo/ent...,8095793,"Área de Gobierno de Políticas Sociales, Famili...",http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28045,PASEO CHOPERA 41,40.394165,-3.699012,Titular del Área: José Fernández Sán

In [4]:
key_places_location = pd.DataFrame({
    "id": key_places["id"],
    'latitude': key_places['location.latitude'],
    'longitude': key_places['location.longitude']
})
key_places_location

,id,latitude,longitude
0,12046758,40.398796,-3.584448
1,11867687,40.394160,-3.699021
2,12040036,40.415265,-3.711579
3,12040015,40.415265,-3.711579
4,12040068,40.415199,-3.710611
...,...,...,...
1231,213,40.419110,-3.696280
1232,8095780,40.419110,-3.696280
1233,8095793,40.394165,-3.699012
1234,212,40.461975,-3.617672


In [5]:
key_places_location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         1236 non-null   object 
 1   latitude   1233 non-null   float64
 2   longitude  1233 non-null   float64
dtypes: float64(2), object(1)
memory usage: 29.1+ KB


In [6]:
key_places.isnull()
key_places.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   @id                             1236 non-null   object 
 1   id                              1236 non-null   object 
 2   title                           1236 non-null   object 
 3   relation                        1236 non-null   object 
 4   address.district.@id            1232 non-null   object 
 5   address.area.@id                1221 non-null   object 
 6   address.locality                1235 non-null   object 
 7   address.postal-code             1236 non-null   object 
 8   address.street-address          1236 non-null   object 
 9   location.latitude               1233 non-null   float64
 10  location.longitude              1233 non-null   float64
 11  organization.organization-desc  1236 non-null   object 
 12  organization.accesibility       12

In [7]:
data_bicimad = pd.read_csv("../data/bicimad_stations.csv", sep="\t")
data_bicimad


,Unnamed: 0,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates
0,0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]"
1,1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]"
2,2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]"
3,3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]"
4,4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,259,265,257 - INEF,0,257,Avenida Juan de Herrera frente a la calle Paul...,1,0,24,3,19,0,Point,"[-3.72997, 40.43896]"
260,260,266,258 - Ciudad Universitaria 1,2,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,9,14,0,Point,"[-3.72699, 40.44375]"
261,261,267,259 - Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,10,0,Point,"[-3.72693, 40.44342]"
262,262,268,260 - Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,12,10,0,Point,"[-3.7272945, 40.4483322]"


In [8]:
data_bicimad[['Longitude', 'Latitude']] = data_bicimad['geometry.coordinates'].str.strip('[]').str.split(',', expand=True).astype("float64")
data_bicimad

,Unnamed: 0,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates,Longitude,Latitude
0,0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]",-3.701834,40.417214
1,1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]",-3.701603,40.417313
2,2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]",-3.705842,40.420589
3,3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]",-3.706917,40.430294
4,4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]",-3.702587,40.428552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,259,265,257 - INEF,0,257,Avenida Juan de Herrera frente a la calle Paul...,1,0,24,3,19,0,Point,"[-3.72997, 40.43896]",-3.729970,40.438960
260,260,266,258 - Ciudad Universitaria 1,2,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,9,14,0,Point,"[-3.72699, 40.44375]",-3.726990,40.443750
261,261,267,259 - Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,10,0,Point,"[-3.72693, 40.44342]",-3.726930,40.443420
262,262,268,260 - Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,12,10,0,Point,"[-3.7272945, 40.4483322]",-3.727295,40.448332


In [9]:
data_bicimad_location = pd.DataFrame({
    "id": data_bicimad["id"],
    'latitude': data_bicimad['Latitude'],
    'longitude': data_bicimad['Longitude']
})
data_bicimad_location

,id,latitude,longitude
0,1,40.417214,-3.701834
1,2,40.417313,-3.701603
2,3,40.420589,-3.705842
3,4,40.430294,-3.706917
4,5,40.428552,-3.702587
...,...,...,...
259,265,40.438960,-3.729970
260,266,40.443750,-3.726990
261,267,40.443420,-3.726930
262,268,40.448332,-3.727295


In [10]:
data_bicimad_location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         264 non-null    int64  
 1   latitude   264 non-null    float64
 2   longitude  264 non-null    float64
dtypes: float64(2), int64(1)
memory usage: 6.3 KB


In [11]:
data_bicipark = pd.read_csv("../data/bicipark_stations.csv", sep=";")
data_bicipark

,Unnamed: 0,stationId,stationName,address,city,zip_code,total_places,free_places,enabled,reserved_places,geometry.type,geometry.coordinates
0,0,74,Bicipark Fuente de la Mora,Estación de Fuente de la Mora,Madrid,28050,10,9,1,0,Point,"[-3.6630679, 40.4846838]"
1,1,22,Bicipark Orense,"Avenida General Perón, 27",Madrid,28020,15,9,1,1,Point,"[-3.693897, 40.452973]"
2,2,49,Bicipark Montalban,"Calle de Montalbán, 5",Madrid,28014,16,15,1,0,Point,"[-3.690786, 40.418146]"
3,3,12,Bicipark Almagro,"Calle Almagro, 11",Madrid,28010,16,12,1,0,Point,"[-3.692879, 40.430361]"
4,4,9,Bicipark Recoletos,"Paseo de Recoletos, 2",Madrid,28001,10,3,0,0,Point,"[-3.692245, 40.420354]"
5,5,5,Bicipark Recuerdo,"Calle Hiedra, 26",Madrid,28036,13,13,1,0,Point,"[-3.678921, 40.472419]"
6,6,25,Bicipark Salamanca,Pl. del Marqués de Salamanca,Madrid,28006,5,-1,1,0,Point,"[-3.679172, 40.430315]"
7,7,67,Bicipark Villa de Paris,"Calle de Orellana, 23, 21",Madrid,28004,10,8,1,1,Point,"[-3.693698, 40.425608]"
8,8,1154,Bicipark Pitis,"Calle Pau Arroyo del Fresno, 181",Madrid,28049,10,7,1,0,Point,"[-3.7249142, 40.4959425]"
9,9,1169,Bicipark Aviación,Parking Aviación Española,Madrid,28024,10,10,1,0,Point,"[-3.78369522, 40.383491516]"


In [12]:
data_bicipark[['Longitude', 'Latitude']] = data_bicipark['geometry.coordinates'].str.strip('[]').str.split(',', expand=True).astype("float64")

In [13]:
data_bicipark

,Unnamed: 0,stationId,stationName,address,city,zip_code,total_places,free_places,enabled,reserved_places,geometry.type,geometry.coordinates,Longitude,Latitude
0,0,74,Bicipark Fuente de la Mora,Estación de Fuente de la Mora,Madrid,28050,10,9,1,0,Point,"[-3.6630679, 40.4846838]",-3.663068,40.484684
1,1,22,Bicipark Orense,"Avenida General Perón, 27",Madrid,28020,15,9,1,1,Point,"[-3.693897, 40.452973]",-3.693897,40.452973
2,2,49,Bicipark Montalban,"Calle de Montalbán, 5",Madrid,28014,16,15,1,0,Point,"[-3.690786, 40.418146]",-3.690786,40.418146
3,3,12,Bicipark Almagro,"Calle Almagro, 11",Madrid,28010,16,12,1,0,Point,"[-3.692879, 40.430361]",-3.692879,40.430361
4,4,9,Bicipark Recoletos,"Paseo de Recoletos, 2",Madrid,28001,10,3,0,0,Point,"[-3.692245, 40.420354]",-3.692245,40.420354
5,5,5,Bicipark Recuerdo,"Calle Hiedra, 26",Madrid,28036,13,13,1,0,Point,"[-3.678921, 40.472419]",-3.678921,40.472419
6,6,25,Bicipark Salamanca,Pl. del Marqués de Salamanca,Madrid,28006,5,-1,1,0,Point,"[-3.679172, 40.430315]",-3.679172,40.430315
7,7,67,Bicipark Villa de Paris,"Calle de Orellana, 23, 21",Madrid,28004,10,8,1,1,Point,"[-3.693698, 40.425608]",-3.693698,40.425608
8,8,1154,Bicipark Pitis,"Calle Pau Arroyo del Fresno, 181",Madrid,28049,10,7,1,0,Point,"[-3.7249142, 40.4959425]",-3.724914,40.495942
9,9,1169,Bicipark Aviación,Parking Aviación Española,Madrid,28024,10,10,1,0,Point,"[-3.78369522, 40.383491516]",-3.783695,40.383492


In [14]:
data_bicipark_location = pd.DataFrame({
    "id": data_bicimad["id"],
    'latitude': data_bicimad['Latitude'],
    'longitude': data_bicimad['Longitude']
})
data_bicipark_location

,id,latitude,longitude
0,1,40.417214,-3.701834
1,2,40.417313,-3.701603
2,3,40.420589,-3.705842
3,4,40.430294,-3.706917
4,5,40.428552,-3.702587
...,...,...,...
259,265,40.438960,-3.729970
260,266,40.443750,-3.726990
261,267,40.443420,-3.726930
262,268,40.448332,-3.727295


In [15]:
from shapely.geometry import Point
import geopandas as gpd

# Methods

def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair points in degrees 
    # (e.g.: Start Point -> 40.4400607 / -3.6425358 End Point -> 40.4234825 / -3.6292625)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

In [16]:
distance_meters([key_places_location["latitude"][0], key_places_location["longitude"][0],data_bicimad_location["latitude"][i], data_bicimad_location["longitude"][i]for i in range(len(data_bicimad_location)])



SyntaxError: invalid syntax (1503176961.py, line 1)

In [17]:
def nearest(x):
    dist = [distance_meters(key_places['location.latitude'][x],key_places['location.longitude'][x] , data_bicimad['Latitude'][i], data_bicimad['Longitude'][i])[0] for i in range(len(data_bicimad['Longitude']))]
    return f'Your nearest station is {data_bicimad["name"][dist.index(min(dist))].split("- ")[1]} located at {round(min(dist), 2)} meters from you'
nearest(78)
      

'Your nearest station is Embajadores 191 located at 3067.47 meters from you'

In [18]:
def nearestbicimad2_input(title):
    index = key_places[key_places['title']==title].index.item()
    dist = [distance_meters(key_places['location.latitude'][index],key_places['location.longitude'][index], data_bicimad['Latitude'][i], data_bicimad['Longitude'][i])[0] for i in range(len(data_bicimad))]
    nearest_index = dist.index(min(dist))
    result =  data_bicimad.loc[nearest_index]
    result_df = pd.DataFrame([result], columns=result.index)
    return result_df
nearestbicimad2_input("Grupo Municipal Socialista de Madrid")

,Unnamed: 0,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates,Longitude,Latitude
36,36,40,36 - Plaza Ramales,2,36,Plaza de Ramales,1,0,24,10,13,0,Point,"[-3.71183, 40.4168]",-3.71183,40.4168


In [19]:
def nearestbicimad_input(title):
    index = key_places[key_places['title']==title].index.item()
    dist = [distance_meters(key_places['location.latitude'][index],key_places['location.longitude'][index], data_bicimad['Latitude'][i], data_bicimad['Longitude'][i])[0] for i in range(len(data_bicimad))]
    nearest_index = dist.index(min(dist))
    result =  data_bicimad.loc[nearest_index]
    resultbicimad_df = pd.DataFrame([result], columns=result.index)
    return resultbicimad_df



nearestbicimad_input("Grupo Municipal Socialista de Madrid")

,Unnamed: 0,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates,Longitude,Latitude
36,36,40,36 - Plaza Ramales,2,36,Plaza de Ramales,1,0,24,10,13,0,Point,"[-3.71183, 40.4168]",-3.71183,40.4168


In [20]:
def nearestbicipark_input(title):
    index = key_places[key_places['title']==title].index.item()
    dist = [distance_meters(key_places['location.latitude'][index],key_places['location.longitude'][index], data_bicipark['Latitude'][i], data_bicipark['Longitude'][i])[0] for i in range(len(data_bicipark))]
    nearest_index = dist.index(min(dist))
    result =  data_bicipark.loc[nearest_index]
    resultbicipark_df = pd.DataFrame([result], columns=result.index)
    return resultbicipark_df

nearestbicipark_input("Grupo Municipal Socialista de Madrid")

,Unnamed: 0,stationId,stationName,address,city,zip_code,total_places,free_places,enabled,reserved_places,geometry.type,geometry.coordinates,Longitude,Latitude
10,10,1238,Bicipark Canalejas,-,Madrid,-,10,10,1,0,Point,"[-3.6994506, 40.417807676]",-3.699451,40.417808


In [22]:
def nearestbicimad_input(title):
    index = key_places[key_places['title']==title].index.item()
    dist = [distance_meters(key_places['location.latitude'][index],key_places['location.longitude'][index], data_bicimad['Latitude'][i], data_bicimad['Longitude'][i])[0] for i in range(len(data_bicimad))]
    nearest_index = dist.index(min(dist))
    result =  data_bicimad.loc[nearest_index]
    resultbicimad_df = pd.DataFrame([result], columns=result.index)
    resultbicimad_df.to_csv('./output/resultbicimadinput.csv', index=False)
    return f'Your nearest station is {data_bicimad["name"][dist.index(min(dist))].split("- ")[1]} located at {round(min(dist), 2)} meters from you and has {data_bicimad["dock_bikes"][dist.index(min(dist))]} bikes and {data_bicimad["free_bases"][dist.index(min(dist))]} free bases available. The CSV is ready in the Output folder.'

nearestbicimad_input("Grupo Municipal Socialista de Madrid")

'Your nearest station is Plaza Ramales located at 173.15 meters from you and has 10 bikes and 13 free bases available. The CSV is ready in the Output folder.'

In [21]:
def nearestbicipark_input(title):
    index = key_places[key_places['title']==title].index.item()
    dist = [distance_meters(key_places['location.latitude'][index],key_places['location.longitude'][index], data_bicipark['Latitude'][i], data_bicipark['Longitude'][i])[0] for i in range(len(data_bicipark))]
    nearest_index = dist.index(min(dist))
    result =  data_bicipark.loc[nearest_index]
    resultbicipark_df ["Place of interest" == title, "Type of Place" == "Instalaciones accessible","StationName", "adress", "city"]= pd.DataFrame([result], columns=result.index)
    resultbicipark_df.to_csv('./output/resultbiciparkinput.csv', index=False)
    return f'Your nearest station is {data_bicipark["stationName"][dist.index(min(dist))]} located at {round(min(dist), 2)} meters from you and has {data_bicipark["free_places"][dist.index(min(dist))]} free bases available. The CSV is ready in the Output folder.'
nearestbicipark_input("Grupo Municipal Socialista de Madrid")

NameError: name 'resultbicipark_df' is not defined

In [23]:

def nearestbicimad_input(title):
    index = key_places[key_places['title']==title].index.item()
    dist = [distance_meters(key_places['location.latitude'][index],key_places['location.longitude'][index], data_bicimad['Latitude'][i], data_bicimad['Longitude'][i])[0] for i in range(len(data_bicimad))]
    nearest_index = dist.index(min(dist))
    result =  data_bicimad.loc[nearest_index]
    resultbicimad_df = pd.DataFrame([result], columns=result.index)
    resultbicimad_df.to_csv('./output/resultbicimadinput.csv', index=False)
    return f'Your nearest station is {data_bicimad["name"][dist.index(min(dist))].split("- ")[1]} located at {round(min(dist), 2)} meters from you. The CSV is ready in the Output folder.'
nearestbicimad_input("Grupo Municipal Socialista de Madrid")

'Your nearest station is Plaza Ramales located at 173.15 meters from you. The CSV is ready in the Output folder.'

In [24]:
def nearest_station(x):
    dist = [distance_meters(key_places['location.latitude'][x],key_places['location.longitude'][x] , data_bicimad['Latitude'][i], data_bicimad['Longitude'][i])[0] for i in range(len(data_bicimad['Longitude']))]
    return data_bicimad["name"][dist.index(min(dist))].split("- ")[1]

nearest_station(1)
      

'Entrada Matadero'

In [25]:
def nearest_distance(x):
    dist = [distance_meters(key_places['location.latitude'][x],key_places['location.longitude'][x] , data_bicimad['Latitude'][i], data_bicimad['Longitude'][i])[0] for i in range(len(data_bicimad['Longitude']))]
    return round(min(dist), 2)
nearest_distance(1036)

5554.4

In [31]:
nearest(1)

'Your nearest station is Entrada Matadero located at 215.44 meters from you'

In [29]:
station_coords = data_bicimad[['Latitude', 'Longitude']].to_numpy()

def nearest_station(row):
    # Calculate distances using vectorized operations
    distances = cdist([(row['location.latitude'], row['location.longitude'])], station_coords)
    min_index = np.argmin(distances)
    nearest_station_name = data_bicimad["name"].iloc[min_index].split("- ")[1]
    return nearest_station_name

# Apply the function to the DataFrame
key_places['nearest_station'] = key_places.apply(nearest_station, axis=1)






In [40]:
import geopandas as gpd
from shapely.geometry import Point

def to_mercator(lat, long):
    # Transform a single latitude/longitude point to pseudo-mercator coordinates
    point = gpd.GeoSeries([Point(long, lat)], crs=4326)  # Note: The order is longitude, latitude
    point_mercator = point.to_crs(3857)
    return point_mercator

def distance_meters(start_point, finish_point):
    # Calculate the distance between two points in pseudo-mercator coordinates
    return start_point.distance(finish_point).iloc[0]

# Precompute Mercator GeoSeries for each station
station_coords_mercator = [to_mercator(lat, lon) for lat, lon in station_coords]

def nearest_distance(row):
    # Transform the current row's coordinates to a Mercator GeoSeries
    row_mercator = to_mercator(row['location.latitude'], row['location.longitude'])

    # Calculate distances using the distance_meters function
    distances = [distance_meters(row_mercator, station) for station in station_coords_mercator]

    # Find the nearest distance
    nearest_distance = min(distances)

    # Return the nearest distance rounded to 2 decimal places
    return round(nearest_distance, 2)

# Apply the function to the DataFrame
key_places['nearest_distances'] = key_places.apply(nearest_distance, axis=1)


In [41]:
key_places

,@id,id,title,relation,address.district.@id,address.area.@id,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,organization.organization-desc,organization.accesibility,organization.schedule,organization.services,organization.organization-name,@type,nearest_station,nearest_distances
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,12046758,Centro de acogida 'Juan Luis Vives',http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28052,CALLE ALCALDE JUAN DE MATA SEVILLANO 18,40.398796,-3.584448,Información Complementaria: Plazas: 140...,1,"Servicio ininterrumpido, 24 horas los 365 días...","Alojamiento, Manutención: desayuno, comida, me...",Centro de acogida 'Juan Luis Vives',NaN,Pavones,5586.20
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,11867687,Departamento de Protección a la Infancia y Ado...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28045,PASEO CHOPERA 41 PLANTA Baja,40.394160,-3.699021,"Metro: Legazpi (línea 3 y 6)Bus: 6, 8, 18, ...",1,De lunes a viernes de 9 a 14 horas.,,Departamento de Protección a la Infancia y Ado...,https://datos.madrid.es/egob/kos/entidadesYorg...,Entrada Matadero,246.88
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,12040036,Grupo Municipal Más Madrid,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28013,CALLE MAYOR 71 PLANTA 1ª y 2ª,40.415265,-3.711579,Portavoz: Rita Maestre Fernández Portavoz adj...,1,,,Grupo Municipal Más Madrid,https://datos.madrid.es/egob/kos/entidadesYorg...,Plaza Ramales,209.62
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,12040015,Grupo Municipal Socialista de Madrid,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28013,CALLE MAYOR 71 PLANTA 3ª y 4ª,40.415265,-3.711579,Portavoz: M&amp;ordf; Reyes Maroto Illera Por...,1,"De 9 a 14 h, previa cita concertada en el telé...",,Grupo Municipal Socialista de Madrid,https://datos.madrid.es/egob/kos/entidadesYorg...,Plaza Ramales,209.62
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,12040068,Grupo Municipal del Partido Popular,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28005,PLAZA VILLA 5,40.415199,-3.710611,Portavoz: Carlos Izquierdo Torres Portavoz a...,1,,,Grupo Municipal del Partido Popular,https://datos.madrid.es/egob/kos/entidadesYorg...,Plaza del Cordón,136.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,https://datos.madrid.es/egob/catalogo/tipo/ent...,213,Área de Gobierno de Hacienda y Personal,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28014,CALLE ALCALA 45,40.419110,-3.696280,Titular del Área: Engracia Hidalgo Tena Com...,1,,,Área de Gobierno de Hacienda y Personal,https://datos.madrid.es/egob/kos/entidadesYorg...,Banco de España A,92.92
1232,https://datos.madrid.es/egob/catalogo/tipo/ent...,8095780,Área de Gobierno de Obras y Equipamientos,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28014,CALLE ALCALA 45 PLANTA 3,40.419110,-3.696280,Titular del Área: Paloma García Romero. Comp...,1,,,Área de Gobierno de Obras y Equipamientos,https://datos.madrid.es/egob/kos/entidadesYorg...,Banco de España A,92.92
1233,https://datos.madrid.es/egob/catalogo/tipo/ent...,8095793,"Área de Gobierno de Políticas Sociales, Famili...",http://www.madrid.es/sites/v/index.jsp?v

In [55]:
import pandas as pd
import numpy as np
import geopandas as gpd
from scipy.spatial.distance import cdist
from shapely.geometry import Point

# Assuming data_bicimad and key_places are already defined DataFrames

# For nearest_station function
station_coords = data_bicimad[['Latitude', 'Longitude']].to_numpy()

def nearest_bicimad_direction(row):
    distances = cdist([(row['location.latitude'], row['location.longitude'])], station_coords)
    min_index = np.argmin(distances)
    nearest_station_adress = data_bicimad["address"].iloc[min_index]
    return nearest_station_adress

key_places['Station Location'] = key_places.apply(nearest_bicimad_direction, axis=1)

def nearest_station(row):
    distances = cdist([(row['location.latitude'], row['location.longitude'])], station_coords)
    min_index = np.argmin(distances)
    nearest_station_name = data_bicimad["name"].iloc[min_index].split("- ")[1]
    return nearest_station_name

key_places['Bicimad station'] = key_places.apply(nearest_station, axis=1)

# For nearest_distance function
def to_mercator(lat, long):
    point = gpd.GeoSeries([Point(long, lat)], crs=4326)
    point_mercator = point.to_crs(3857)
    return point_mercator

def distance_meters(start_point, finish_point):
    return start_point.distance(finish_point).iloc[0]

station_coords_mercator = [to_mercator(lat, lon) for lat, lon in station_coords]

def nearest_distance(row):
    row_mercator = to_mercator(row['location.latitude'], row['location.longitude'])
    distances = [distance_meters(row_mercator, station) for station in station_coords_mercator]
    nearest_distance = min(distances)
    return f'{round(nearest_distance, 1)} meters'

key_places['Station Distance'] = key_places.apply(nearest_distance, axis=1)

# Selecting and renaming specific columns
final_df = key_places[['title', "address.street-address", 'Bicimad station','Station Location', 'Station Distance']].copy()
final_df['address.street-address'] = final_df['address.street-address'].apply(lambda x: x.title())
final_df.rename(columns={'title': 'Place of Interest', 
                         'address.street-address': 'Place address'}, inplace=True)


# Adding a constant column
final_df['Type of place'] = 'Instalaciones accesibles municipales'

# Export to CSV
csv_file_path = './output/resultkeyplacesbicimaddataframe.csv'   # Replace with your desired path
final_df.to_csv(csv_file_path, index=False)


In [58]:
import pandas as pd
import numpy as np
import geopandas as gpd
from scipy.spatial.distance import cdist
from shapely.geometry import Point

# Assuming data_bicipark and key_places are already defined DataFrames

# For nearest_station function
station_coords = data_bicipark[['Latitude', 'Longitude']].to_numpy()

def nearest_bicipark_direction(row):
    distances = cdist([(row['location.latitude'], row['location.longitude'])], station_coords)
    min_index = np.argmin(distances)
    nearest_station_adress = data_bicipark["address"].iloc[min_index]
    return nearest_station_adress

key_places['Station Location'] = key_places.apply(nearest_bicipark_direction, axis=1)

def nearest_station(row):
    distances = cdist([(row['location.latitude'], row['location.longitude'])], station_coords)
    min_index = np.argmin(distances)
    nearest_station_name = data_bicipark["stationName"].iloc[min_index]
    return nearest_station_name

key_places['Bicipark station'] = key_places.apply(nearest_station, axis=1)

# For nearest_distance function
def to_mercator(lat, long):
    point = gpd.GeoSeries([Point(long, lat)], crs=4326)
    point_mercator = point.to_crs(3857)
    return point_mercator

def distance_meters(start_point, finish_point):
    return start_point.distance(finish_point).iloc[0]

station_coords_mercator = [to_mercator(lat, lon) for lat, lon in station_coords]

def nearest_distance(row):
    row_mercator = to_mercator(row['location.latitude'], row['location.longitude'])
    distances = [distance_meters(row_mercator, station) for station in station_coords_mercator]
    nearest_distance = min(distances)
    return f'{round(nearest_distance, 1)} meters'

key_places['Station Distance'] = key_places.apply(nearest_distance, axis=1)

# Selecting and renaming specific columns
final_df = key_places[['title', "address.street-address", 'Bicipark station', 'Station Location', 'Station Distance']].copy()
final_df['address.street-address'] = final_df['address.street-address'].apply(lambda x: x.title())
final_df.rename(columns={'title': 'Place of Interest', 
                         'address.street-address': 'Place address'}, inplace=True)

# Adding a constant column
final_df['Type of place'] = 'Instalaciones accesibles municipales'

# Export to CSV
csv_file_path = './output/resultkeyplacesbiciparkdataframe.csv'   # Replace with your desired path
final_df.to_csv(csv_file_path, index=False)


In [66]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair points in degrees 
    # (e.g.: Start Point -> 40.4400607 / -3.6425358 End Point -> 40.4234825 / -3.6292625)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

def nearestbicimad_input(title):
    index = key_places[key_places['title']==title].index.item()
    dist = [distance_meters(key_places['location.latitude'][index],key_places['location.longitude'][index], data_bicimad['Latitude'][i], data_bicimad['Longitude'][i])[0] for i in range(len(data_bicimad))]
    nearest_index = dist.index(min(dist))
    result =  data_bicimad.loc[nearest_index]
    resultbicimad_df = pd.DataFrame([result], columns=result.index)
    finalresultbicimad_df = resultbicimad_df[['name', "address", 'dock_bikes']].copy()

    finalresultbicimad_df.rename(columns={'name': 'Bicimad Station', 
                         'address': 'Station address',
                        'dock_bikes': 'Bikes available'}, inplace=True)
    finalresultbicimad_df.to_csv('./output/resultbicimadinput.csv', index=False)
    return f'Your nearest station is {data_bicimad["name"][dist.index(min(dist))].split("- ")[1]} located at {round(min(dist), 2)} meters from you and has {data_bicimad["dock_bikes"][dist.index(min(dist))]} bikes and {data_bicimad["free_bases"][dist.index(min(dist))]} free bases available. The CSV is ready in the Output folder.'

nearestbicimad_input("Grupo Municipal Socialista de Madrid")

'Your nearest station is Plaza Ramales located at 173.15 meters from you and has 10 bikes and 13 free bases available. The CSV is ready in the Output folder.'